In this notebook, we apply transfer learning on VGG-16 network and solve MNSIT digit classifiation task very rapidly.

# Transfer learning - Fixed Feature Extractor


## Creating and using a Fixed Feature Extractor


In [ ]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
mean = [0.1307]
std = [0.3081]

transform =transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.ToTensor(),
   transforms.Normalize((0.1307,), (0.3081,)),
   transforms.Lambda(lambda x: x.repeat(3, 1, 1))
   
])
trainset = datasets.MNIST(root='~/.pytorch/MNIST',train=True, download=True,transform=transform)
testset = datasets.MNIST(root='~/.pytorch/MNIST',train=False, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.pytorch/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.pytorch/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.pytorch/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.pytorch/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST/MNIST/raw



In [ ]:
model = models.vgg16(pretrained=True)
model.features

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [ ]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
for param in model.parameters():
  param.requires_grad=False

In [ ]:
model.classifier[-1]=nn.Sequential(nn.Linear(in_features=4096, out_features=10),nn.LogSoftmax(dim=1))
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=10, bias=True)
    (1): LogSoftmax(dim=1)
  )
)

In [ ]:
criterion=nn.NLLLoss()

## Training the Fixed Feature Extractor

In [ ]:
from torch.optim import Adam

model = model.to(device)
optimizer = Adam(model.parameters())

In [ ]:
num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)})  Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/938)  Batch loss : 2.355513572692871
Epoch(0/1 : Batch number(2/938)  Batch loss : 2.3121888637542725
Epoch(0/1 : Batch number(3/938)  Batch loss : 2.2674596309661865
Epoch(0/1 : Batch number(4/938)  Batch loss : 2.202890157699585
Epoch(0/1 : Batch number(5/938)  Batch loss : 1.9799960851669312
Epoch(0/1 : Batch number(6/938)  Batch loss : 1.870864748954773
Epoch(0/1 : Batch number(7/938)  Batch loss : 1.8888568878173828
Epoch(0/1 : Batch number(8/938)  Batch loss : 1.8657402992248535
Epoch(0/1 : Batch number(9/938)  Batch loss : 1.5870283842086792
Epoch(0/1 : Batch number(10/938)  Batch loss : 1.6474915742874146
Epoch(0/1 : Batch number(11/938)  Batch loss : 1.647162914276123
Epoch(0/1 : Batch number(12/938)  Batch loss : 1.483595371246338
Epoch(0/1 : Batch number(13/938)  Batch loss : 1.4254322052001953
Epoch(0/1 : Batch number(14/938)  Batch loss : 1.4069381952285767
Epoch(0/1 : Batch number(15/938)  Batch loss : 1.2985435724258423
Epoch(0/1 : Batch number

## Evaluating thet network and viewing images

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

## Accuracy of the model

In [ ]:
images, labels = images.cuda(), labels.cuda() # add this line

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0


    for batch, (images, labels) in enumerate(testloader,1):
        images, labels = images.cuda(), labels.cuda() # add this line

        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 91.875% 
